In [64]:
import sys, os
sys.path.append(os.path.dirname(os.path.abspath(os.path.dirname("PP.py"))))

from PP import NLP_PreProcess

In [65]:
# 학습 기반 토크나이저 : Soynlp
# 품사 태깅, 단어 토큰화 등을 지원하는 단어 토크나이저
# 비지도 학습으로 단어 토큰화 : 데이터에 자주 등장하는 단어들을 단어로 분석
# 내부적으로 단어 점수 표로 동작

In [66]:
# !pip install soynlp
# https://github.com/lovit/soynlp

In [67]:
from konlpy.tag import Okt

tokenizer = Okt()

print(tokenizer.morphs('에이비식스 이대휘 1월 최애돌 기부 요정'))

## 형태소 분석 시 매개변수 stem=True 설정
print(tokenizer.morphs('에이비식스 이대휘 1월 최애돌 기부 요정 입니다', stem=True))

print(tokenizer.morphs("에이비식스 이대휘 1월 최애돌 기부 요정", norm=True))

['에이', '비식스', '이대', '휘', '1월', '최애', '돌', '기부', '요정']
['에이', '비식스', '이대', '휘', '1월', '최애', '돌', '기부', '요정', '이다']
['에이', '비식스', '이대', '휘', '1월', '최애', '돌', '기부', '요정']


In [68]:
# [Sonlpy]사용 => 말뭉치 데이터셋으로 

In [69]:
from urllib.request import urlretrieve
import os
filename = "text_data.txt"

if not os.path.exists(path=filename):
    urlretrieve("https://raw.githubusercontent.com/lovit/soynlp/master/tutorials/2016-10-20.txt", filename)

In [70]:
## 학습 데이터 처리
from soynlp import DoublespaceLineCorpus
from soynlp.word import WordExtractor

In [71]:
## 훈련 데이터 문서 분리
corpus = DoublespaceLineCorpus(filename)
print(f"훈련 데이터 문서 : {len(corpus)}개")

훈련 데이터 문서 : 30091개


In [72]:
### Sonlpy 학습 진행
word_extractor = WordExtractor()
# 학습 진행하여 단어별 점수
word_extractor.train(corpus)
# 단어별 점수표 추출
word_score_table = word_extractor.extract()

training was done. used memory 1.823 Gb
all cohesion probabilities was computed. # words = 223348
all branching entropies was computed # words = 361598
all accessor variety was computed # words = 361598


In [73]:
# 단어별 점수표 확인
for idx, key in enumerate(iterable = word_score_table.keys()):
    print(f'[{idx}] - {key}')
    if idx==30: break

[0] - 될
[1] - 싶
[2] - 뻘
[3] - 5
[4] - 뉴
[5] - 역
[6] - 봐
[7] - 궐
[8] - 균
[9] - 낱
[10] - 랍
[11] - 찔
[12] - 콩
[13] - 7
[14] - 팰
[15] - 윽
[16] - 능
[17] - 컷
[18] - 톈
[19] - 클
[20] - 뱀
[21] - 쐈
[22] - 빕
[23] - 향
[24] - 핏
[25] - 접
[26] - ㅇ
[27] - 끓
[28] - 누
[29] - 그
[30] - 절


In [74]:
# 응집확률(cohesion probability) : 내부 문자열(substring)이 얼마나 응집하여 자주 등장하는지를 판단하는 척도

#    - 원리 : 문자열을 문자 단위로 분리, 왼쪽부터 순서대로 문자를 추가
#             각 문자열이 주어졌을 때 그 다음 문자가 나올 확률을 계산/ 누적곱 한 값

#    - 값이 높을수록 : 전체 코퍼스에서 이 문자열 시퀸스는 하나의 단어로 등장할 가능성 높음

In [75]:
word_score_table['바다'].cohesion_forward # 일반 명사만 나올 확률을 낮다 

0.06393648140409527

In [76]:
word_score_table['바다에'].cohesion_forward # "바다에"처럼 명사 + 조사가 나올 확률을 상대적으로 높다 

0.11518621707955429

In [77]:
word_score_table['바'].cohesion_forward # 얘만 나올 확률은 0이다

0

In [78]:
# SOYNLP의 L tokenizer
# - 띄어쓰기 단위로 나눈 어절 토큰 : L토큰 + R토큰
# (예 : '공원에' -> '공원' + '에',  '공부하는' -> '공부' + '하는')
# 분리 기준 : 점수가 가장 높은 L토큰을 찾아내는 원리

In [79]:
from soynlp.tokenizer import LTokenizer

scores = {word:score.cohesion_forward for word, score in word_score_table.items()} # 응집확률이 높게 되도록 뽑아낸다 
l_tokenizer = LTokenizer(scores=scores) # 그냥 쪼개지 말고, LTokenizer로 쪼개자! 
l_tokenizer.tokenize("국제사회와 우리의 노력들로 범죄를 척결하자", flatten=False)

[('국제사회', '와'), ('우리', '의'), ('노력', '들로'), ('범죄', '를'), ('척결', '하자')]

In [80]:
# 최대 점수 토크나이저
# 띄어쓰기가 되지 않는 문장에서 점수가 높은 글자 시퀸스를 순차적으로 찾아내는 토크나이저
# 띄어쓰기가 되어 있지 않은 문장을 넣어서 점수를 통해 토큰화 된 결과
from soynlp.tokenizer import MaxScoreTokenizer

maxscore_tokenizer = MaxScoreTokenizer(scores=scores)
maxscore_tokenizer.tokenize("국제사회와우리의노력들로범죄를척결하자")

['국제사회', '와', '우리', '의', '노력', '들로', '범죄', '를', '척결', '하자']

In [81]:
word_score_table['국'].cohesion_forward ,word_score_table['국제'].cohesion_forward ,word_score_table['국제사'].cohesion_forward ,word_score_table['국제사회'].cohesion_forward ,word_score_table['국제사회와'].cohesion_forward # "와"가 생기자마자 확률이 떨어지므로, 여기서 끊어주는 것이다 

(0,
 0.07856876882976202,
 0.09217735975351507,
 0.20075093164820865,
 0.17387399904982392)

In [1]:
# SOYNLP를 이용한 반복되는 문자 정제
# ㅋㅋ, ㅎㅎ 등의 이모티콘인 경우 불필요하게 연속되는 경우 많음
# ㅋㅋ, ㅋㅋㅋ, ㅋㅋㅋㅋ와 같은 경우를 모두 서로 다른 단어로 처리하는 것은 불필요
# >> 반복되는 것은 하나로 정규화 

from soynlp.normalizer import *

print(emoticon_normalize("앜ㅋㅋㅋㅋㅋㅋ이영화존잼쓰 ㅠㅠㅠㅠ", num_repeats=1))
print(emoticon_normalize("앜ㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ이영화존잼쓰ㅠㅠㅠㅠㅠㅠㅠㅠ", num_repeats=2))
print(emoticon_normalize("앜ㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ이영화존잼쓰 ㅠㅠㅠㅠㅠㅠㅠㄴㄴㄴㄴㄴㄴ😊😊😊😊😊😊😊😊😊", num_repeats=3))

아ㅋ영화존잼쓰 ㅠ
아ㅋㅋ영화존잼쓰ㅠㅠ
아ㅋㅋㅋ영화존잼쓰 ㅠㅠㅠㄴㄴㄴ😊😊😊😊😊😊😊😊😊


In [83]:
print(repeat_normalize("와하하하하핫", num_repeats=2))

와하하핫


In [84]:
#%pip install customized_konlpy

In [85]:
from ckonlpy.tag import Twitter

twitter = Twitter()
twitter.morphs('은경이는 사무실로 갔습니다.')

c:\ProgramData\anaconda3\envs\NLP\lib\site-packages\konlpy\tag\_okt.py:17: UserWarning: "Twitter" has changed to "Okt" since KoNLPy v0.4.5.
  warn('"Twitter" has changed to "Okt" since KoNLPy v0.4.5.')


['은', '경이', '는', '사무실', '로', '갔습니다', '.']

In [86]:
twitter.add_dictionary(words="은경이", tag="Noun")

In [87]:
twitter.morphs('은경이는 사무실로 갔습니다.')

['은경이', '는', '사무실', '로', '갔습니다', '.']

In [88]:
# 맥캡은 코랩에서 사용할 수 있다 

In [89]:
from konlpy.tag import Komoran

komoran = Komoran()

# 사용자 정의 사전 추가
# 추가할 단어와 그에 해당하는 품사를 리스트로 넣어줍니다.
user_dictionary = [('사용자단어', '사용자품사'), ('추가할단어', '추가할품사')]

# 사용자 정의 사전 적용
for word, pos in user_dictionary:
    print(word, pos)
    komoran.tagset.setdefault(word, pos)

# 예시 문장 형태소 분석
result = komoran.morphs("사용자단어를 추가할 수 있습니다.")
print(result)


사용자단어 사용자품사
추가할단어 추가할품사
['사용자', '단어', '를', '추가', '하', 'ㄹ', '수', '있', '습니다', '.']


In [90]:
komoran.tagset

{'EC': '연결 어미',
 'EF': '종결 어미',
 'EP': '선어말어미',
 'ETM': '관형형 전성 어미',
 'ETN': '명사형 전성 어미',
 'IC': '감탄사',
 'JC': '접속 조사',
 'JKB': '부사격 조사',
 'JKC': '보격 조사',
 'JKG': '관형격 조사',
 'JKO': '목적격 조사',
 'JKQ': '인용격 조사',
 'JKS': '주격 조사',
 'JKV': '호격 조사',
 'JX': '보조사',
 'MAG': '일반 부사',
 'MAJ': '접속 부사',
 'MM': '관형사',
 'NA': '분석불능범주',
 'NF': '명사추정범주',
 'NNB': '의존 명사',
 'NNG': '일반 명사',
 'NNP': '고유 명사',
 'NP': '대명사',
 'NR': '수사',
 'NV': '용언추정범주',
 'SE': '줄임표',
 'SF': '마침표, 물음표, 느낌표',
 'SH': '한자',
 'SL': '외국어',
 'SN': '숫자',
 'SO': '붙임표(물결,숨김,빠짐)',
 'SP': '쉼표,가운뎃점,콜론,빗금',
 'SS': '따옴표,괄호표,줄표',
 'SW': '기타기호 (논리수학기호,화폐기호)',
 'VA': '형용사',
 'VCN': '부정 지정사',
 'VCP': '긍정 지정사',
 'VV': '동사',
 'VX': '보조 용언',
 'XPN': '체언 접두사',
 'XR': '어근',
 'XSA': '형용사 파생 접미사',
 'XSN': '명사파생 접미사',
 'XSV': '동사 파생 접미사',
 '사용자단어': '사용자품사',
 '추가할단어': '추가할품사'}